In [1]:
import snowflake.snowpark as snowpark
from snowflake.snowpark import Session
from dotenv import load_dotenv
import os

load_dotenv()


connection_parameters = {
   "account": os.getenv('account_snow'),
   "user": os.getenv('user_snow'),
   "password":os.getenv('password_snow'),
   "role": "ACCOUNTADMIN",
   "database": "NEW_DB", 
   "schema": "PUBLIC"
  
}  

session = Session.builder.configs(connection_parameters).create()

In [2]:
employee_data = [
    [1,'TONY',24000,101],
    [2,'STEVE',17000,101],
    [3,'BRUCE',9000,101],
    [4,'WANDA',20000,102],
    [5,'VICTOR',12000,102],
    [6,'STEPHEN',10000,103],
    [7,'HANK',15000,103],
    [8,'THOR',21000,103]
]

employee_schema = ["EMP_ID", "EMP_NAME", "SALARY", "DEPT_ID"]

df_emp =session.createDataFrame(employee_data, schema=employee_schema)
df_emp.show()

------------------------------------------------
|"EMP_ID"  |"EMP_NAME"  |"SALARY"  |"DEPT_ID"  |
------------------------------------------------
|1         |TONY        |24000     |101        |
|2         |STEVE       |17000     |101        |
|3         |BRUCE       |9000      |101        |
|4         |WANDA       |20000     |102        |
|5         |VICTOR      |12000     |102        |
|6         |STEPHEN     |10000     |103        |
|7         |HANK        |15000     |103        |
|8         |THOR        |21000     |103        |
------------------------------------------------



In [3]:
from snowflake.snowpark import Window
from snowflake.snowpark.functions import row_number, desc, col, min

#// creating a WindowSpec
windowSpec = Window.partitionBy("DEPT_ID").orderBy(col("SALARY").desc())

In [4]:
df_emp.withColumn("RANK", row_number().over(windowSpec)).show()

---------------------------------------------------------
|"EMP_ID"  |"EMP_NAME"  |"SALARY"  |"DEPT_ID"  |"RANK"  |
---------------------------------------------------------
|4         |WANDA       |20000     |102        |1       |
|5         |VICTOR      |12000     |102        |2       |
|1         |TONY        |24000     |101        |1       |
|2         |STEVE       |17000     |101        |2       |
|3         |BRUCE       |9000      |101        |3       |
|8         |THOR        |21000     |103        |1       |
|7         |HANK        |15000     |103        |2       |
|6         |STEPHEN     |10000     |103        |3       |
---------------------------------------------------------



In [5]:
df_emp.select("*", row_number().over(windowSpec).alias("RANK")).show()

---------------------------------------------------------
|"EMP_ID"  |"EMP_NAME"  |"SALARY"  |"DEPT_ID"  |"RANK"  |
---------------------------------------------------------
|4         |WANDA       |20000     |102        |1       |
|5         |VICTOR      |12000     |102        |2       |
|1         |TONY        |24000     |101        |1       |
|2         |STEVE       |17000     |101        |2       |
|3         |BRUCE       |9000      |101        |3       |
|8         |THOR        |21000     |103        |1       |
|7         |HANK        |15000     |103        |2       |
|6         |STEPHEN     |10000     |103        |3       |
---------------------------------------------------------



In [6]:
df_emp.withColumn("RANK", row_number().over(windowSpec)).filter(col("RANK")==1).sort("DEPT_ID").show()

---------------------------------------------------------
|"EMP_ID"  |"EMP_NAME"  |"SALARY"  |"DEPT_ID"  |"RANK"  |
---------------------------------------------------------
|1         |TONY        |24000     |101        |1       |
|4         |WANDA       |20000     |102        |1       |
|8         |THOR        |21000     |103        |1       |
---------------------------------------------------------



SELECT * FROM(
  SELECT
    EMP_ID, EMP_NAME, SALARY, DEPT_ID,
    ROW_NUMBER() OVER (PARTITION BY DEPT_ID ORDER BY SALARY DESC) AS RANK
  FROM EMPLOYEES
)
WHERE RANK = 1 
ORDER BY DEPT_ID ;

In [9]:
windowSpec = Window.partitionBy("DEPT_ID")

In [ ]:
df_emp.withColumn("TOTAL_SAL", sum("SALARY").over(windowSpec)).show()

SELECT
   EMP_ID, EMP_NAME, SALARY, DEPT_ID,
   SUM(SALARY) OVER (PARTITION BY DEPT_ID) AS TOTAL_SAL
FROM EMPLOYEES
;

In [11]:
windowSpec = Window.partitionBy("DEPT_ID").orderBy(col("EMP_ID"))


In [ ]:
df_emp.withColumn("CUM_SAL", sum("SALARY").over(windowSpec)).show()